In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import json
from tqdm import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-val/examples.json
/kaggle/input/model_update_val/pytorch/default/1/model/adapter_model.safetensors
/kaggle/input/model_update_val/pytorch/default/1/model/adapter_config.json
/kaggle/input/model_update_val/pytorch/default/1/model/README.md
/kaggle/input/model_update_val/pytorch/default/1/model/tokenizer.json
/kaggle/input/model_update_val/pytorch/default/1/model/tokenizer_config.json
/kaggle/input/model_update_val/pytorch/default/1/model/special_tokens_map.json
/kaggle/input/vina7b_lora_3epoch/pytorch/default/1/model/adapter_model.safetensors
/kaggle/input/vina7b_lora_3epoch/pytorch/default/1/model/adapter_config.json
/kaggle/input/vina7b_lora_3epoch/pytorch/default/1/model/README.md
/kaggle/input/vina7b_lora_3epoch/pytorch/default/1/model/tokenizer.json
/kaggle/input/vina7b_lora_3epoch/pytorch/default/1/model/tokenizer_config.json
/kaggle/input/vina7b_lora_3epoch/pytorch/default/1/model/special_tokens_map.json
/kaggle/input/test-set-data/examples.json
/kaggle/input/d

In [2]:
!pip install transformers accelerate peft bitsandbytes datasets huggingface_hub trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.7 MB/s eta 0:00:00


In [3]:
import os
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

In [4]:
model_name = "/kaggle/input/vina7b_lora_3epoch/pytorch/default/1/model"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # ✅ Tự động sử dụng cả 2 GPU
    torch_dtype = torch.float16
)
# 🔹 Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
print("✅ Mô hình đã tải thành công!")

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

✅ Mô hình đã tải thành công!


In [5]:
import re

In [6]:
import json

with open("/kaggle/input/data-test/examples.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

In [7]:
import re

def generate_query(question, history=[]):
    # Định dạng lại lịch sử hội thoại
    history_text = "\n".join(history) if history else ""

    # Xây dựng input text
    input_text = f"""### Instruction:
Tạo truy vấn Logical Form để lấy thông tin tương ứng với câu hỏi đã cho.

### Input:
Question: {question}

### History:
{history_text}

### Output:
"""

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

    output_ids = model.generate(
        input_ids, 
        max_new_tokens=96,
        num_beams=5,
        num_beam_groups=1,  
        num_return_sequences=5,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature = 0.95,
        top_p = 0.7,
        repetition_penalty = 1.0,
        length_penalty = 1.0
    )
    
    def clean_output(text):
        text = text.split("Output:")[-1].strip()  
        text = re.split(r"\n+", text)[0]  # Lấy dòng đầu tiên
        return text

    output_texts = [clean_output(tokenizer.decode(out, skip_special_tokens=True)) for out in output_ids]
    
    return output_texts

In [8]:
query1 = generate_query("Nữ diễn viên nào là người lồng tiếng cho bộ phim South Park và được thuê làm ca sĩ?")
query2 = generate_query("Trên hồ Winnipeg có những hồ nào trên sông?")
query3 = generate_query("ai đã kết hôn với diễn viên của Bepanaah?")

print("📌 Truy vấn 1:", query1)
print("📌 Truy vấn 2:", query2)
print("📌 Truy vấn 3:", query3)

📌 Truy vấn 1: ['( AND ( JOIN ( R [ diễn viên lồng tiếng ] ) [ South Park ] ) ( JOIN [ nghề nghiệp ] [ ca sĩ ] ) )', '( AND ( JOIN ( R [ diễn viên lồng tiếng ] ) [ South Park ] ) ( JOIN [ người biểu diễn ] [ Justin Timberlake ] ) )', '( AND ( JOIN ( R [ diễn viên lồng tiếng ] ) [ South Park ] ) ( JOIN [ người biểu diễn ] [ Cher ] ) )', '( JOIN ( R [ diễn viên lồng tiếng ] ) ( AND ( JOIN ( R [ diễn viên lồng tiếng ] ) [ South Park ] ) ( JOIN [ nghề nghiệp ] [ ca sĩ ] ) ) )', '( AND ( JOIN ( R [ diễn viên lồng tiếng ] ) [ South Park ] ) ( JOIN [ nghề nghiệp ] [ ca sĩ nhạc đồng quê ] ) )']
📌 Truy vấn 2: ['( JOIN ( R [ phụ lưu ] ) ( JOIN ( R [ tọa lạc trong hoặc bên cạnh thủy vực ] ) [ Winnipeg ] ) )', '( JOIN ( R [ tọa lạc trong hoặc bên cạnh thủy vực ] ) ( JOIN ( R [ cửa sông ] ) [ Sông Winnipeg ] ) )', '( JOIN ( R [ tọa lạc trong hoặc bên cạnh thủy vực ] ) [ Winnipeg ] ) ( JOIN ( R [ phụ lưu ] ) [ Winnipeg ] )', '( JOIN ( R [ tọa lạc trong hoặc bên cạnh thủy vực ] ) ( JOIN ( R [ phụ lưu 

In [ ]:
from tqdm import tqdm

output_data = []

# ✅ Duyệt qua từng câu hỏi và tạo truy vấn logic với beam search
for data in tqdm(json_data):
    question = data["input"]
    predicted_query = generate_query(question)
    output_data.append({
        "question": question,
        "predicted_query": predicted_query,
        "gen_label": data["output"]
    })

# ✅ Ghi kết quả vào file JSON
output_file = "/kaggle/working/generated_predictions_test_3ep.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=4, ensure_ascii=False)

print(f"✅ Dự đoán hoàn tất! Kết quả được lưu tại: {output_file}")

  0%|          | 8/4450 [00:30<4:38:37,  3.76s/it]